In [1]:
import requests
import lxml.html
import pprint

from delete_database_entries import *

## Note: start your local server!

In the django project directory (where the manage.py file is located) type: "python manage.py runserver"

## Deleting all entries in the database to make a clean start

To keep the database clean we delete all content first :O

In [3]:
delete_models("sessions")

Deleting: sessions
104


## You can get an overview of the installed pipelines and their configurations

In [6]:
with requests.Session() as s:
    response = s.get('http://localhost:8000/pipelineschemas/')
    response_data = response.json()
    pp.pprint(response_data)
    #pp.pprint(response_data["pipelineschemas"]["LGPPP"]["schema"]["required"])

{   'pipelineschemas': {   'LGPPP': {   'id': 'LGPPP',
                                        'label': 'LOFAR GRID Pre-Processing '
                                                 'Pipeline',
                                        'schema': {   'description': 'This is '
                                                                     'the '
                                                                     'LOFAR '
                                                                     'GRID '
                                                                     'Pre-Processing '
                                                                     'Pipeline. '
                                                                     'Here we '
                                                                     'print a '
                                                                     'description '
                                                                     'of the '
   

## We can make a session object

Here imagine that the session object is an object that will actually run your pipeline configuration on your data.

In [7]:
data = {
        "email": "pipo@popo.com",
        "description": "",
        "pipeline" : "LGPPP",
        "config": "{\"avg_freq_step\": 1, \"avg_time_step\": 1, \"do_demix\": 1, \"demix_freq_step\": 1, \"demix_time_step\": 1, \"demix_sources\": 1, \"select_nl\": 1,\"parset\": 1}",
        "observation": "somenumber",
        }

In [8]:
with requests.Session() as s:
    response = s.post('http://localhost:8000/sessions/', data=data)
    print("Response code: ", response.status_code)
    pp.pprint(response.json())

Response code:  201
{   'config': {   'avg_freq_step': 1,
                  'avg_time_step': 1,
                  'demix_freq_step': 1,
                  'demix_sources': 1,
                  'demix_time_step': 1,
                  'do_demix': 1,
                  'parset': 1,
                  'select_nl': 1},
    'date_created': '2018-07-19T13:46:01.994799Z',
    'date_modified': '2018-07-19T13:46:02.150600Z',
    'description': '',
    'email': 'pipo@popo.com',
    'id': 105,
    'observation': 'somenumber',
    'pipeline': 'LGPPP',
    'pipeline_respone': 'pipeline response LGPPP',
    'pipeline_version': '0.0',
    'status': 'started'}


## Now lets have a look at the content of the database through some get requests

### Sessions:

In [9]:
with requests.Session() as s:
    response = s.get('http://localhost:8000/sessions/')
    pp.pprint(response.json())

[   {   'config': {   'avg_freq_step': 1,
                      'avg_time_step': 1,
                      'demix_freq_step': 1,
                      'demix_sources': 1,
                      'demix_time_step': 1,
                      'do_demix': 1,
                      'parset': 1,
                      'select_nl': 1},
        'date_created': '2018-07-19T13:46:01.994799Z',
        'date_modified': '2018-07-19T13:46:02.150600Z',
        'description': '',
        'email': 'pipo@popo.com',
        'id': 105,
        'observation': 'somenumber',
        'pipeline': 'LGPPP',
        'pipeline_respone': 'pipeline response LGPPP',
        'pipeline_version': '0.0',
        'status': 'started'}]
